## Implementing Data Warehouse Security in Microsoft Fabric

#### Create a new "Patients" table in the Data Warehouse

In [ ]:
CREATE TABLE dbo.Patients
(   
    PatientID INT NOT NULL,   
    FirstName VARCHAR(50) MASKED WITH (FUNCTION = 'partial(1,"XXXXXXX",0)') NULL,     
    LastName VARCHAR(50) NOT NULL,     
    Phone VARCHAR(20) MASKED WITH (FUNCTION = 'default()') NULL,     
    Email VARCHAR(50) MASKED WITH (FUNCTION = 'email()') NULL,   
    DateOfBirth DATE MASKED WITH (FUNCTION = 'default()') NULL,
    MedicalRecordNumber VARCHAR(20) MASKED WITH (FUNCTION = 'partial(0,"XXXXXXXXXX",4)') NOT NULL
);
   
INSERT INTO dbo.Patients (PatientID, FirstName, LastName, Phone, Email, DateOfBirth, MedicalRecordNumber) VALUES
(1001, 'John', 'Doe', '555-123-4567', 'john.doe@example.com', '1985-06-15', 'MRN-1234567890'),
(1002, 'Jane', 'Smith', '444-987-6543', 'jane.smith@example.com', '1990-09-22', 'MRN-0987654321'),
(1003, 'Robert', 'Brown', '333-456-7890', 'robert.brown@example.com', '1978-03-10', 'MRN-5678901234');

SELECT * FROM dbo.Patients;


#### Grant UNMASK to the 2nd User with "Viewer" Role assigned for the workspace

In [ ]:
 GRANT UNMASK ON dbo.Patients TO [<username>@<your_domain>.com];

#### Create a Salary Table for Row-Level Security (RLS) Implementation

In [ ]:
CREATE TABLE dbo.Salary  
(  
    EmployeeID INT,  
    EmployeeEmail VARCHAR(60),  
    Position VARCHAR(50),  
    Salary DECIMAL(10,2) MASKED WITH (FUNCTION = 'default()')  
);
    
-- Populate the table with 6 rows of data
INSERT INTO dbo.Salary (EmployeeID, EmployeeEmail, Position, Salary) VALUES
(101, 'employee1@company.com', 'Software Engineer', 85000.00),   
(102, 'employee2@company.com', 'Data Scientist', 92000.00),   
(103, 'employee3@company.com', 'Project Manager', 105000.00),  
(104, 'employee4@company.com', 'DevOps Engineer', 98000.00),   
(105, 'employee5@company.com', 'Business Analyst', 88000.00),   
(106, 'employee6@company.com', 'HR Manager', 75000.00);  
    
SELECT * FROM dbo.Salary;  


#### Create a Security Predicate Function and a Security Policy for RLS (Row-Level Security)

In [ ]:
--Create a separate schema to hold the row-level security objects (the predicate function and the security policy)
CREATE SCHEMA rls;
GO
   
/*Create the security predicate defined as an inline table-valued function.
A predicate evaluates to true (1) or false (0). This security predicate returns 1,
meaning a row is accessible, when a row in the EmployeeEmail column is the same as the user
executing the query.*/   
--Create a function to evaluate who is querying the table
CREATE FUNCTION rls.fn_securitypredicate(@EmployeeEmail AS VARCHAR(60)) 
    RETURNS TABLE  
WITH SCHEMABINDING  
AS  
    RETURN SELECT 1 AS fn_securitypredicate_result   
WHERE @EmployeeEmail = USER_NAME();
GO   
/*Create a security policy to invoke and enforce the function each time a query is run on the Salary table.
The security policy has a filter predicate that silently filters the rows available to 
read operations (SELECT, UPDATE, and DELETE). */
CREATE SECURITY POLICY SalaryFilter 
ADD FILTER PREDICATE rls.fn_securitypredicate(EmployeeEmail)   
ON dbo.Salary  
WITH (STATE = ON);
GO

#### Denying 2nd user the permission to view the "MedicalRecordNumber" column in the Patients table

In [ ]:
DENY SELECT ON dbo.Patients (MedicalRecordNumber) TO [username@your_domain.com];